In [ ]:
# see link
# https://github.com/Fraud-Detection-Handbook

package ='05-sklearn.tree'
name='Decision Tree'
tuningAndParameters='03-Random Undersampling'

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import datamanagement as dm
reload(dm)

import result as resultMd
reload(resultMd)

import graph as gf
reload(gf)

print('done')

In [ ]:
dfLearning, dfValidation =dm.getDataLearningAndValidation()

dfLearning.head()

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

TEST_SIZE = 0.20 # test size using_train_test_split
RANDOM_STATE = 0

predictors = dm.getPredictors(dfLearning)
target = dm.getTarget()

x_train0, x_test, y_train0, y_test = train_test_split(dfLearning[predictors], dfLearning[target], test_size = TEST_SIZE, 
                                                        stratify=dfLearning[target],
                                                        random_state = RANDOM_STATE)
rate=60
undersample = RandomUnderSampler(sampling_strategy=1/rate,random_state=42)
x_train, y_train = undersample.fit_resample(x_train0, y_train0)

In [ ]:
%%script false

from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from datetime import datetime
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
import numpy as np
from sklearn.metrics import f1_score

train_f1s=[]
test_f1s =[]
range= []

x_train0, x_test, y_train0, y_test = train_test_split(dfLearning[predictors], dfLearning[target], test_size = TEST_SIZE, 
                                                        stratify=dfLearning[target],
                                                        random_state = RANDOM_STATE)

rates = np.arange(5,100,5)
for rate in rates:
    print("",rate,1/rate)
    undersample = RandomUnderSampler(sampling_strategy=1/rate,random_state=42)
    x_train, y_train = undersample.fit_resample(x_train0, y_train0)

    print(x_train0.shape)
    print(x_train.shape)
    #print(y_train.value_counts())
    
    modelClf = DecisionTreeClassifier(random_state=42)
    parameters={'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 2, 'min_samples_split': 16}
    modelClf.set_params(**parameters)

    modelClf.fit(x_train, y_train)
    predsTrain = modelClf.predict(x_train)
    predsTest = modelClf.predict(x_test)

    train_f1=f1_score(y_train, predsTrain)
    print("f1 train {:.4f}".format(train_f1))
    
    test_f1=f1_score(y_test, predsTest)
    print("f1 test  {:.4f}".format(test_f1))
    
    train_f1s.append(train_f1)
    test_f1s.append(test_f1)
    range.append(rate)
    print('-----------------------')

dm.plt_train_test(range, train_f1s, "train", test_f1s," test")

In [ ]:
%%script false
dm.plt_train_test(range, train_f1s, "train", test_f1s," test")

In [ ]:
x_train0, x_test, y_train0, y_test = train_test_split(dfLearning[predictors], dfLearning[target], test_size = TEST_SIZE, 
                                                        stratify=dfLearning[target],
                                                        random_state = RANDOM_STATE)

rate = 60
undersample = RandomUnderSampler(sampling_strategy=1/rate,random_state=42)
x_train, y_train = undersample.fit_resample(x_train0, y_train0)
print(x_train0.shape)
print(x_train.shape)

In [ ]:
%%script false

from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

#{'criterion': 'entropy', 'max_depth': 9, 'min_samples_leaf': 7, 'min_samples_split': 32
dic_param={
    'criterion':["gini","entropy"],
    'max_depth': randint(2,12),
    'min_samples_leaf': randint(2,10),
    'min_samples_split': randint(10,20)
}
modelClf = DecisionTreeClassifier(random_state=42)

random_search = RandomizedSearchCV(modelClf,dic_param, scoring='f1', verbose=10,cv=4).fit(x_train, y_train)
print(random_search.best_params_)
print(random_search.best_score_)


#{'criterion': 'entropy', 'max_depth': 11, 'min_samples_leaf': 7, 'min_samples_split': 11}
#0.7646951430633323



In [ ]:
%%script false

from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint
from sklearn.model_selection import GridSearchCV

#{'criterion': 'entropy', 'max_depth': 9, 'min_samples_leaf': 7, 'min_samples_split': 32
dic_param={
    'criterion':["gini","entropy"],
    'max_depth': [3,4,5,6,7,8,9,10,11,12],
    'min_samples_leaf': [2,3,4,5,6,7,8,9],
    'min_samples_split': [8,10,11,12,13,14,16]
}
modelClf = DecisionTreeClassifier(random_state=42)

random_search = GridSearchCV(modelClf,dic_param, scoring='f1', verbose=10,cv=4).fit(x_train, y_train)
print(random_search.best_params_)
print(random_search.best_score_)

#{'criterion': 'entropy', 'max_depth': 11, 'min_samples_leaf': 7, 'min_samples_split': 11}
#0.7646951430633323

#{'criterion': 'entropy', 'max_depth': 11, 'min_samples_leaf': 3, 'min_samples_split': 14}
#0.782934333385938


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from datetime import datetime

modelClf = DecisionTreeClassifier(random_state=42)
parameters={'criterion': 'entropy', 'max_depth': 11, 'min_samples_leaf': 3, 'min_samples_split': 14}

modelClf.set_params(**parameters)
then= datetime.now()
modelClf.fit(x_train, y_train)
now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)
resultMd.update_time_response_result(package, name, tuningAndParameters, duration_in_s)

predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

F1Learning =f1_score(y_train, predsTrain)
F1Test=f1_score(y_test, predsTest)
dm.show_confusion_matrix(y_train, predsTrain,'Confusion matrix learning data')
print(f"f1 train {F1Learning:.3f}")
dm.show_confusion_matrix(y_test, predsTest,'Confusion matrix test data')
print(f"f1 test {F1Test:.3f}")
resultMd.update_learning_test_result(package, name, tuningAndParameters, F1Learning,F1Test)

In [ ]:
gf.show_importance(modelClf, predictors)

In [ ]:
gf.show_prediction_graph(modelClf, x_test,y_test)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
# Plot the decision tree
plt.figure(figsize=(20, 10))
plot_tree(modelClf, filled=True, feature_names=x_train.columns, class_names=['Geniune', 'Fraudulent'])
plt.show()

In [ ]:
predsValidation = modelClf.predict(dfValidation[predictors])
f1=f1_score(dfValidation[target], predsValidation)

dm.show_confusion_matrix(dfValidation[target], predsValidation,'Confusion matrix validation data')
print(f"f1 validation {f1:.3f}")
resultMd.update_performance_result(package, name, tuningAndParameters, f1)